In [5]:
import numpy as np
import pickle
import cvxopt
from active_set import quadratic_problem

### Get Quadratic Problem

In [18]:
EXAMPLE_NUM = 6

In [19]:
with open(f"Data/problem{EXAMPLE_NUM}.pickle", "rb") as f:
    E, Q_diag, q, b, u, solution = pickle.load(f)
Q = np.diag(Q_diag)
E = E.toarray()
E, b = E[:-1], b[:-1]              # linear independence
u = np.nan_to_num(u, posinf=1e6)   # np.inf doesn't work with cvxopt

C:\Users\Wallace\AppData\Local\Temp\ipykernel_16916\1344267124.py:2: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  E, Q_diag, q, b, u, solution = pickle.load(f)


In [20]:
E, Q_diag, q, b, u, solution

(array([[-1., -1., -1., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., -1.,  0.,  0.]]),
 array([1., 1., 1., 4., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 3., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 4., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 3., 1., 4., 1., 1.,
        1., 1., 3., 1., 1., 1., 1., 4., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 4., 1., 1.]),
 array([1.e+05, 1.e+05, 1.e+05, 3.e+00, 1.e+05, 1.e+05, 1.e+05, 1.e+05,
        1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05,
        1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 1.e+05, 2.e+00,
        1

In [21]:
n, m = E.shape
print(f"#nodes:{n+1}, #edges:{m}")

#nodes:20, #edges:108


### Init and solve with cvxopt

In [22]:
P_cvxopt = cvxopt.matrix(2*Q)
p_cvxopt = cvxopt.matrix(q)
# x >= 0  <>  G = -I,  h = 0
# x <= u  <>  G =  I,  h = u
G_cvxopt = cvxopt.matrix(np.block([
    [-np.eye(m)], 
    [np.eye(m)]
]))
h_cvxopt = cvxopt.matrix(np.concatenate((np.zeros(m), u)))
# Ex = b
A_cvxopt = cvxopt.matrix(E)
b_cvxopt = cvxopt.matrix(b)

sol = cvxopt.solvers.qp(P_cvxopt, p_cvxopt, # cost
                        G_cvxopt, h_cvxopt, # ineq. constraints
                        A_cvxopt, b_cvxopt) # eq. constraints

     pcost       dcost       gap    pres   dres
 0: -1.5032e+11 -5.0328e+10  3e+11  1e+04  1e-16
 1: -1.7781e+10 -8.7781e+08  7e+10  2e+03  3e-13
 2: -1.4053e+08 -1.3746e+07  3e+09  9e+01  1e-14
 3:  2.4666e+06 -1.3071e+07  5e+07  9e-01  3e-16
 4:  1.5036e+06 -3.5608e+06  8e+06  9e-02  2e-16
 5:  8.2600e+05 -1.5025e+06  3e+06  4e-02  3e-16
 6:  1.8135e+05 -3.6546e+05  7e+05  6e-03  3e-16
 7:  4.5840e+04 -6.9566e+04  1e+05  1e-03  2e-16
 8:  1.6588e+03  3.9909e+02  2e+03  1e-05  2e-16
 9:  1.1749e+03  1.1623e+03  2e+01  1e-07  2e-16
10:  1.1700e+03  1.1699e+03  2e-01  1e-09  2e-16
11:  1.1700e+03  1.1700e+03  2e-03  1e-11  2e-16
12:  1.1700e+03  1.1700e+03  2e-05  1e-13  2e-16
Optimal solution found.


In [23]:
1/2* solution @ Q @ solution + solution @ q

657.5

In [24]:
assert(sol["status"] == 'optimal')
sol_cvxopt = np.array(sol["x"]).reshape(m)
1/2* sol_cvxopt @ Q @ sol_cvxopt + sol_cvxopt @ q

657.5000048893501

In [25]:
np.max(sol_cvxopt - solution)

3.1752378504279477e-12

### Init and solve with our method 
(only works on problem 1)

In [26]:
class min_flow(quadratic_problem):
    def __init__(self, Q, E, b, q, l, u, verbose=False):
        n, m = E.shape
        M = np.zeros((n,n))
        super().__init__(E, b, q, 2*Q, M, l=l, u=u, verbose=verbose)

qp = min_flow(Q, E, b, q, np.zeros(m), u, verbose=False)

qp.solve()

[[-1. -1. -1. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ... -1.  0.  0.]]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.]
[-1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[ 0. -1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[ 0. -1.  0.  0.  

C:\Users\Wallace\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Wallace\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


In [27]:
qp.x

array([nan, nan, nan,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [28]:
qp.get_solution()

nan


AssertionError: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]